In [12]:
# download the necessary packages of NLP
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [9]:
# import the file
import pandas as pd
Data = pd.read_csv('/content/drive/MyDrive/smsspamcollection/SMSSpamCollection',sep='\t',names = ["label","messeges"])
# Data fils is not a comma seperated one but tab seperated.so here we define seperation in tab and we give titles to each columns of data.
Data.head()
# ham indicates not spam and spam indicates spam messages.


,label,messeges
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
# Text cleaning or preprocessing:
import nltk
import re
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0,len(Data)):
  review = re.sub('[^a-zA-z]', ' ', Data['messeges'][i])                # to remove elements in paragraph other than a-z ,A-Z case words.
  review = review.lower()                                              # Lowerize the sentences by stoping keywords such as 'he', 'it' and split it into words
  review = review.split()
  review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [36]:
# create Tf-Idf model for corpus
from sklearn.feature_extraction.text import TfidfVectorizer
Tfid = TfidfVectorizer()
X = Tfid.fit_transform(corpus).toarray()
X.shape

(5572, 7103)

In [34]:
# Convert label class for the convenience
y=pd.get_dummies(Data['label'])
y.head()

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [35]:
# It is not advisable to get hold of two colounm labels.Here the importance is given to spam label.In spam colounm 0 indicates no spam and 1 indicates spam.
# So due to enough information at spam column ..it is considered as stand alone label array.
y = y.iloc[:,1].values

In [38]:
# Training model using Naive bayes classifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [43]:
# Evaluate the mmodel:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
y_pred=spam_detect_model.predict(X_test)
print(accuracy_score(y_test,y_pred)*100)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

97.21973094170404
[[955   0]
 [ 31 129]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       955
           1       1.00      0.81      0.89       160

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

